# VISUALIZATION CONCENPT FOR SYNDROMIC DATA

1. Discharge Trends
    
    a. Figures summariseing patient  discharged (dead,referred, alive) by month or March April May
    
2. Patient Distibution by Age and Gender for the regions and month
3. Priority Indicators (Datasets recorded by health providers)

    a. COVID-19 vaccination status (period - March,April,Mya). \
        Recorded or not, Indeterminate,No, Undocumented,Yes
    b. HIV Status
        Negative,Positive,Undocumented,Unknown
    
    c. Comobordity at Admission
        Undocumented, No, Yes,
4. Duration of Hospital Stay
    In days , number of days, interquatile range, by period
5. Presenting symptoms at Admission
    Chest pain.cpugh,diarrhoea, difficulty in breathing, fever, headache, vomiting
6. Completion of Demographic measures
7. Recording of Vital Signs
8. Recording of Key Priority Measures
9. Laboratory tests done at admission (count and disease/condition testsd)
10. Disease patterns at discharge
11. Outcome at Discharge


    

## Notes

There exists ages below 13 and some above 100. The current age graph is filtered between 0 - 100 years. A total of 992 above 70 years.

In [1]:
%cd '/Users/jmwanga/Library/CloudStorage/OneDrive-KemriWellcomeTrust/Visualization/Kenya-COVID-19-Dashboard/syndromic_data/'

/Users/jmwanga/Library/CloudStorage/OneDrive-KemriWellcomeTrust/Visualization/Kenya-COVID-19-Dashboard/syndromic_data


In [55]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
pd.options.mode.chained_assignment = None
import numpy as np
import sys
sys.path.append("utils.py")
import utils
from utils import *

In [14]:

data = pd.read_csv('./Syndromic_Surveillance_data2.csv',low_memory=False)
data[['date_of_admission', 'date_of_discharge']] = data[['date_of_admission', 'date_of_discharge']].\
                                apply(pd.to_datetime, format="%Y-%m-%d")
data[(data['sex'] != 'Male') & (data['sex'] != 'Female')]
data = data[data['date_of_admission'] >'2022-12-31']
# data.age_years.fillna(data.calculated_age,inplace=True)
data['month'] = data['date_of_admission'].apply(lambda x: x.strftime('%b-%y'))
data['age_years'] = data['age']
#map regions


map_dict = {'Naivasha':'Central',
'Kiambu':'Central','Mbagathi':'Central','Mama_Lucy':'Central','JOOTRH':'Western','Kisumu':'Western','Kakamega':'Western','Busia':'Western',
'Kitale':'Western','Bungoma':'Western','Kisii':'Western'}

map_dict2 = {'Jaramogi Oginga Odinga Teaching and Referral Hospital':'Western','Naivasha County Referral Hospital':'Central',
                'Kiambu Level 5 Hospital':'Central','Machakos Level 5 Hospital':'Central',
                'Mama Lucy Kibaki Hospital':'Central','Mbagathi Hospital':'Central',
                'Kerugoya County Referral Hospital':'Central','Kisumu County Hospital':'Western',
                'Kakamega County General & Teaching Referral Hospital':'Western','Busia County Referral Hospital':'Western',
                'Kitale County Referral Hospital':'Western','Bungoma County Hospital':'Western',
                'Kisii Teaching and Referral Hospital':'Western','Coast General Teaching and Referral Hospital':'Coast'}

data['region'] = data['hospital'].map(map_dict2)
#group age groups
def age_group(dataframe,column):
        '''Function to generate age groups. returns age groups column in the data frame'''
        dataframe['age_groups'] = pd.qcut(dataframe[column],
                                       [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,1],
                                       labels=['<10','11-20','21-30','31-40','41-50','51-60','61-70','>70'])
        return dataframe
    
data = age_group(data,'age_years')
data['duration'] = (data['date_of_discharge']-data['date_of_admission'])/np.timedelta64(1,'D')
total_central = len(data[data['region'] == 'Central'])
total_western = len(data[data['region'] == 'Western'])


In [15]:

margin = dict(l=10, r=10, t=5, b=5)

total_patients = len(data)

malep = len(data[data['sex'] == 'Male'])
femalep = len(data[data['sex'] == 'Female'])
#total_patients = malep + femalep
male_percentage = round((malep/(total_patients)*100),1)
female_percentage = round((femalep/(total_patients)*100),1)

titlefont = {"size":11}

reg_data = data[['record_id','date_of_admission','hospital','sex','age_years','region','age_groups','month','outcome',
                 'status_at_discharge','date_of_discharge']]

df_all = data.loc[(data['sex'].isin(['Male','Female'])) & (data['date_of_admission'] >'2022-12-31' )]\
    [['age_groups','month','sex','date_of_admission','calculated_age','age_years']]
    
df_age = df_all[(df_all.age_years > 0) & (df_all.age_years < 100)]
age_sex = df_age.groupby(['sex','age_groups'])[['sex']].count().rename(columns={'sex':'count'}).reset_index()
age_sex_f = age_sex[age_sex['sex'] == 'Female']
age_sex_f['prop'] = round(age_sex_f['count']/age_sex_f['count'].sum()*100,0)

age_sex_m = age_sex[age_sex['sex'] == 'Male']
age_sex_m['prop'] = round(age_sex_m['count']/age_sex_m['count'].sum()*100,0)


In [58]:
titlefont = {"size":11}
def age_gender_plot(male_data,female_data):
    age_plot= go.Figure()
    age_plot.add_trace(go.Bar(y = female_data["age_groups"], x = female_data["prop"]*-1, orientation="h",name = "Female",
                            marker=dict(color=female_color),
                            ))
    age_plot.add_trace(go.Bar(y = male_data["age_groups"], x = male_data["prop"], orientation="h",name = "Male",
                            marker=dict(color=male_color)))
    age_plot.update_layout(barmode = "relative",bargap = 0.5,bargroupgap=0,
                            xaxis =  dict(tickvals = [-60,-40,-20,0,20,40,60],
                                            ticktext = ["60","40","20","0","20","40","60"]),
                            legend = dict(orientation = "h",title=None,yanchor='top',y=1.2,xanchor='right',x=1),
                            margin=margin,paper_bgcolor=plot_color,plot_bgcolor=plot_color),
    age_plot.update_traces(width=0.5)
    age_plot.update_yaxes(tickfont = dict(size=12),title ='Age in Years',title_font = titlefont,ticks='outside',)
    # age_plot.update_xaxes(tickfont = dict(size=12),title = 'Proportion(%)',gridcolor=gridcolor, linecolor=linecolor,ticks='outside',nticks=20)
    return age_plot

age_plot = age_gender_plot(age_sex_f,age_sex_m)

In [63]:
df = reg_data.groupby(['region','sex'])[['sex']].count().rename(columns={'sex':'count'}).reset_index()
df['gender_proportion'] = df['count'].apply(lambda x: x*100/df['count'].sum())


reg_fig = px.bar(df, y='region',x='gender_proportion', color='sex',barmode='group',range_x=([0,100]),
                 color_discrete_sequence= gender_color,text='count')
reg_fig.update_traces(textposition='outside',textfont_size=10)
reg_fig.update_layout(margin=margin,legend=dict(orientation='h',title=None,yanchor='top',y=1,xanchor='right',x=1),
                      uniformtext_minsize=7, uniformtext_mode='hide',paper_bgcolor=plot_color,plot_bgcolor=plot_color)
reg_fig.update_yaxes(tickfont = dict(size=12),title = None,ticks='outside')
reg_fig.update_xaxes(title ='Proportion(%)',nticks=10, tickfont = dict(size=12),linecolor= linecolor,gridcolor=gridcolor,ticks='outside')


reg_period = reg_data.groupby(['region','month'])[['region']].count().rename(columns={'region':'count'}).reset_index()
reg_period['proportion'] = round(100 * reg_period['count']/reg_period.groupby('region')['count'].transform('sum'),1)


def plot_reg_period(data):
    fig = px.bar(data, x='month',y='count',barmode='group',color='region',range_y=[0, data['count'].max()+200],
                        color_discrete_sequence= discrete_color,text = 'count')
    fig.update_xaxes(categoryorder='array',categoryarray=['Jan-23','Feb-23','Mar-23','Apr-23','May-23','Jun-23'])
    fig.update_layout(margin=margin,legend=dict(orientation='h', title=None,yanchor='top',y=1.2,xanchor='right',x=1),
                      paper_bgcolor=plot_color,plot_bgcolor=plot_color)
    fig.update_xaxes(tickfont = dict(size=12),title=None,linecolor='gray',ticks='outside',)
    fig.update_yaxes(tickfont = dict(size=12),nticks=6,title='Frequency',gridcolor=gridcolor,ticks='outside',)
    fig.update_traces(textposition='outside',textfont_size=10)
    
    return fig
reg_per_fig =plot_reg_period(reg_period)


In [100]:

data_campture = data[['record_id','dob_known','date_of_entry','biodata_complete','sex','age_years','region','age_groups','month',\
    'date_of_admission','weight','height','outcome','status_at_discharge','date_of_discharge',
    'd1_present','cause_of_death','temp','heart_rate','resp_rate','blood_pressure_sys','blood_pressure_dia','oxygen_sat']]

data_campture['blood_pressure_dia'] = data_campture['blood_pressure_dia'].fillna(0)
def group_value(column,value):
    grouped = data_campture[data_campture[column].isin([value])][['region', column]].groupby('region').count().reset_index()
    return grouped

def group_between(column,min,max):
    grouped = data_campture[data_campture[column].between(min, max)][['region', column]].groupby('region').count().reset_index()
    return grouped
sex_complete = data_campture[data_campture['sex'].isin(['Male','Female'])][['sex','region']].groupby('region').count().reset_index()
#bio_complete = group_value('biodata_complete','Complete')
# dob_known  = group_value('dob_known','Yes')
temp = group_between('temp',30,45)
heart_rate = group_between('heart_rate',40,150)
resp_rate = group_between('resp_rate',5,30)
height_complete = group_between('height',50,250)
weight_complete = group_between('weight',40,250)
age_complete = group_between('age_years',18,100)
# oxy_rate = group_between('oxygen_sat',20,100)

df_demo = pd.merge(age_complete,sex_complete,on='region').\
    merge(height_complete,on='region').\
        merge(weight_complete,on='region')
            
df_demo.set_index('region',inplace=True)
df_demo = (df_demo.div(total_patients)*100).round(1)
df_vital_signs = pd.merge(temp,resp_rate,on='region').\
    merge(heart_rate,on='region')
                
df_vital_signs.set_index('region',inplace=True)
df_vital_signs = (df_vital_signs.div(total_patients)*100).round(1)

In [103]:
def heatmap_plot(data,yname):
    fig = px.imshow(data,text_auto=True,aspect="auto",range_color=(0,100),
                    color_continuous_scale=px.colors.sequential.amp)#PuBu)amp
    fig.update_layout(margin=margin)
    fig.update_yaxes(title=yname,title_font=titlefont)
    fig.update_xaxes(title=None,side='top', gridcolor='gray',ticks='outside')
    fig.update_coloraxes(colorbar={'title':'Proportion(%)','orientation':'h', 'thickness':7, 'y':-0.4, 'titleside':'bottom'})
    fig.update_traces(ygap=1,xgap=1)

    return fig

fig_demo_heatmap = heatmap_plot(df_demo.T,'Variable')
fig_vital_heatmap = heatmap_plot(df_vital_signs.T,'Vital Signs')




In [109]:
def priority_group(data,column1,column2):
    df = data[[column1,column2]].groupby([column1,column2])[[column2]].count().\
        rename(columns={column2:'count'}).reset_index()
    df['prop'] = round(100*df['count']/df.groupby(column1)['count'].transform('sum'),1)
    return df

hiv_data_region = priority_group(data,'region','hiv_status')
hiv_data_month = priority_group(data,'month','hiv_status')
cov_vaccine_region = priority_group(data,'region','received_covid_19_vaccine')
cov_vaccine_month = priority_group(data,'month','received_covid_19_vaccine')

#fucntion for plot
def priority_plot(data,column,yaxis):
    fig = px.bar(data, x='prop', y=yaxis, color=column,
             orientation='h', color_discrete_map=colors)
    fig.update_xaxes(title='Proportion(%)',title_font=titlefont,gridcolor=gridcolor,ticks='outside')
    fig.update_yaxes(title=yaxis,title_font=titlefont,ticks='outside')
    fig.update_yaxes(categoryorder='array',categoryarray=['Jan-23','Feb-23','Mar-23','Apr-23','May-23','Jun-23'])
    fig.update_layout(barmode='stack',margin=margin,
                      legend=dict(title=None,orientation='h',yanchor='top',y=1.5,x=0))
    return fig
                         
fig_cov_vacc_region = priority_plot(cov_vaccine_region,'received_covid_19_vaccine','region')
fig_cov_vacc_month = priority_plot(cov_vaccine_month,'received_covid_19_vaccine','month')

fig_hiv_region = priority_plot(hiv_data_region,'hiv_status','region')
fig_hiv_period = priority_plot(hiv_data_month,'hiv_status','month')


In [110]:
completion = data[['month','region','biodata_complete','presenting_complaints_and_history_complete','examination_complete',\
    'admission_diagnosis_complete','investigations_complete','discharge_summary_complete']]

def completion_group(column1,column2):
    df = completion[[column1,column2]].groupby([column1,column2])[[column1]].count().\
    rename(columns={column1: 'count'}).reset_index()
    df[(column1+'_'+'prop')] = round(100*df['count']/df.groupby(column2)['count'].transform('sum'),1)
    df.drop('count',axis=1,inplace=True)
    df.set_index('month',inplace=True)
    df.rename(columns={column1:'variable'},inplace=True)
    return df

biodata_month = completion_group('biodata_complete','month')
examination_data_month = completion_group('examination_complete','month')
complaints_history_month = completion_group('presenting_complaints_and_history_complete','month')
admi_diagnosis_month = completion_group('admission_diagnosis_complete','month')
investigations_complete_month = completion_group('investigations_complete','month')
discharge_summary_month = completion_group('discharge_summary_complete','month')


In [127]:
data['dysuria']

0        Empty
1           No
2        Empty
3        Empty
4        Empty
         ...  
19386      NaN
19387      NaN
19388      NaN
19389      NaN
19390      NaN
Name: dysuria, Length: 17986, dtype: object

,record_id,month,region,cough,headache,diarrhoea,chest_pain,vomiting,fever,loss_of_consciousness,dysuria,difficulty_in_breathing,abdominal_pain,easy_fatigability
0,4400001,May-23,Western,Empty,Empty,Empty,Empty,Empty,Empty,Yes,Empty,NaN,Empty,NaN
1,4400002,Mar-23,Western,Empty,Empty,Yes,Empty,No,Empty,No,No,NaN,No,NaN
2,4400003,Mar-23,Western,Yes,Empty,Yes,Empty,Empty,Empty,Empty,Empty,NaN,Empty,NaN
3,4400004,Mar-23,Western,No,Empty,No,Yes,Yes,No,Empty,Empty,NaN,No,NaN
4,4400005,Mar-23,Western,Empty,Yes,Empty,Empty,Empty,Empty,No,Empty,NaN,Empty,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19386,331-163,Mar-23,Western,Empty,Empty,Empty,Empty,Empty,Empty,NaN,NaN,NaN,NaN,NaN
19387,331-164,Feb-23,Western,Empty,Empty,Empty,Empty,Empty,Empty,NaN,NaN,NaN,NaN,NaN
19388,331-165,Feb-23,Western,No,Yes,Empty,No,Empty,Empty,NaN,NaN,NaN,NaN,NaN
19389,331-166,Feb-23,Western,Empty,Yes,No,Yes,No,Empty,NaN,NaN,NaN,NaN,NaN


In [129]:
data.columns

Index(['Unnamed: 0', 'record_id', 'redcap_data_access_group', 'date_of_entry',
       'hospital', 'mar_used', 'ipno', 'control_number', 'subcounty',
       'other_subcounty',
       ...
       'discharged_by_contact', 'discharged_by_cadre', 'completed_by_disch',
       'date_time_completed_disch', 'date_of_outcome', 'age', 'month',
       'region', 'age_groups', 'duration'],
      dtype='object', length=392)

In [121]:
def presenting_symptoms(value):
    sum_len = len(data[value].isin(['Yes','No']))
    df= data[data[value]=='Yes'][['record_id','region',value]]
    df = df.groupby('region')[[value]].count()
    df[value] =  df[value].apply(lambda x: (x/sum_len)*100)
    return df

df_headach = presenting_symptoms('headache')
df_diarrhoea = presenting_symptoms('diarrhoea')
df_cough = presenting_symptoms('cough')
df_chestpain = presenting_symptoms('chest_pain')
df_vomiting = presenting_symptoms('vomiting')
df_fever = presenting_symptoms('fever')
df_vomiting = presenting_symptoms('vomiting')
df_dysuria = presenting_symptoms('dysuria')
df_losscon = presenting_symptoms('loss_of_consciousness')

symp_data = pd.merge(df_headach,df_diarrhoea,on='region').\
    merge(df_cough,on='region').merge(df_chestpain,on='region').\
            merge(df_vomiting,on='region').merge(df_fever,on='region').\
                        merge(df_dysuria,on='region').merge(df_losscon,on='region')

symp_data.rename(columns={'loss_of_consciousness':'LoC'},inplace=True)

df_headach



,headache
region,
Central,6.577338
Coast,0.122317
Western,14.728122


In [114]:
df = symp_data.T
figplot= go.Figure()
figplot.add_trace(go.Bar(x = df['Central']*-1,y=df.index, orientation="h",name = "Central",marker=dict(color=central)))
figplot.add_trace(go.Bar(x = df['Western'],y=df.index, orientation="h",name = "Western",marker=dict(color=western)))
figplot.update_layout(barmode = "relative",bargap = 0.5,bargroupgap=0,
                   xaxis =  dict(tickvals = [-15,-10,-5,0,5,10,15],
                                        ticktext = ['15',"10",'5',"0",'5',"10","15",'20']),
                   legend = dict(orientation = "h",title=None,
                                yanchor='top',y=1.3,xanchor='left',x=0.5),
                   margin=margin,paper_bgcolor=plot_color,plot_bgcolor=plot_color)
figplot.update_traces(width=0.5)
figplot.update_yaxes(tickfont = dict(size=12),title = 'Symptoms',title_font = titlefont,ticks='outside')
figplot.update_xaxes(tickfont = dict(size=12),title = "Proportion(%)",ticks='outside',
                     gridcolor=gridcolor,title_font = titlefont,linecolor=linecolor)


def presenting_symptoms_period(value):
    sum_len = len(data[value].isin(['Yes','No']))
    df= data[data[value]=='Yes'][['record_id','month',value]]
    df = df.groupby('month')[[value]].count()
    df[value] =  round(df[value].apply(lambda x: (100*x/sum_len)),0)
    return df

period_headach = presenting_symptoms_period('headache')
period_diarrhoea = presenting_symptoms_period('diarrhoea')
period_cough = presenting_symptoms_period('cough')
period_chestpain = presenting_symptoms_period('chest_pain')
period_vomiting = presenting_symptoms_period('vomiting')
period_fever = presenting_symptoms_period('fever')
period_vomiting = presenting_symptoms_period('vomiting')
period_dysuria = presenting_symptoms_period('dysuria')
period_losscon = presenting_symptoms_period('loss_of_consciousness')

perioddata= pd.merge(period_headach,period_diarrhoea,on='month').\
    merge(period_cough,on='month').merge(period_chestpain,on='month').\
            merge(period_vomiting,on='month').merge(period_fever,on='month').\
                        merge(period_dysuria,on='month').merge(period_losscon,on='month')


In [115]:

perioddata.rename(columns={'loss_of_consciousness':'LoC'},inplace=True)
perioddata = perioddata.T[['Feb-23','Mar-23','Apr-23','May-23','Jun-23']]

fig_period_heatmap = heatmap_plot(perioddata, 'Symptoms')

count_data = data[['date_of_admission','region']].groupby(['region','date_of_admission'])[['date_of_admission']]\
                         .count().rename(columns={'date_of_admission':'Count'}).reset_index()
count_data['cumsum'] = count_data.groupby('region')['Count'].cumsum()
cum_plot = px.line(count_data, x = 'date_of_admission',y='cumsum',color='region',range_x=['2023-01-01','2023-07-01'],
                   color_discrete_sequence = discrete_color,
                   hover_name='region',hover_data={'region':False,'cumsum':True})

cum_plot.update_xaxes(title = None,linecolor='gray',ticks='outside',dtick='M1',tickformat='%b-%y')
cum_plot.update_yaxes(title = None,ticks='outside')
cum_plot.update_layout(margin = margin,legend = dict(orientation='h',title=None,yanchor='top',y=1.2,xanchor='right',x=1))


In [117]:

##Discharge and Outcom
outcome_sex =  priority_group(data,'outcome','sex')
outcome_region = priority_group(data,'outcome','region')

def plot_figure(data):
    '''Function to plot region for alive and dead'''
    
    fig = px.bar(data, y = 'region',x='prop',color='region',barmode='group',range_x=([0,100]),
                 color_discrete_sequence= discrete_color,text='count')
    fig.update_traces(textposition='outside',textfont_size=10)
    fig.update_layout(margin=margin,legend=dict(orientation='h',title=None,yanchor='top',y=1,xanchor='right',x=1),
                      uniformtext_minsize=7, uniformtext_mode='hide',paper_bgcolor=plot_color,plot_bgcolor=plot_color)
    fig.update_yaxes(tickfont = dict(size=12),title = None,ticks='outside')
    fig.update_xaxes(title ='Proportion(%)',nticks=10, tickfont = dict(size=12),linecolor= linecolor,gridcolor=gridcolor,ticks='outside')
    return fig

##outcome discharge
##STATUS AT DISCHARGE
status_at_discharge = data_campture[['status_at_discharge']].value_counts().rename_axis('status').reset_index(name='counts')
status_at_discharge['prop'] = status_at_discharge['counts']/status_at_discharge['counts'].sum()*100

status_at_discharge_plot = px.bar(status_at_discharge.sort_values('prop'),x='prop',y ='status',orientation='h',text='counts')
                                  
status_at_discharge_plot.update_yaxes(title=None)
status_at_discharge_plot.update_layout(margin=margin,paper_bgcolor=plot_color,plot_bgcolor=plot_color)
status_at_discharge_plot.update_traces(textposition='outside',marker_color='indianred')


############################################################
df_test = data[['outcome','status_at_discharge','date_of_admission','date_of_discharge','sex','age_years','month','region','d1_present','cause_of_death',\
                'disease_specific_tests___1','disease_specific_tests___2','disease_specific_tests___4','disease_specific_tests___5','microbiology___1',
                'biochemistry___3','haematology___1','microbiology___5']]
df_test.rename(columns={'disease_specific_tests___1':'Malaria','disease_specific_tests___2':'TB','disease_specific_tests___4':'HIV',
                        'disease_specific_tests___5':'COVID-19','microbiology___1':'Urinalysis','microbiology___5':'stool_analysis',
                        'biochemistry___3':'Glucose','haematology___1':'Blood Count'},
                inplace=True)

In [118]:
def admission_test_group(data, value):
    '''Function to group data by region or period/Month'''
    data = data[[value,'TB','Malaria','HIV','COVID-19','Urinalysis','Glucose','Blood Count','stool_analysis']]
    data = data.groupby(value).apply(lambda x: x.eq('Checked').sum())
    data = data.drop(value,axis=1)
    data = data.apply(lambda x :round(100*x/5412),1)
    #data = data.T.reset_index()
    return data

test_region = admission_test_group(df_test, 'region')
test_period = admission_test_group(df_test, 'month')
test_period = test_period.T[['Jan-23','Feb-23','Mar-23','Apr-23','May-23','Jun-23']]

test_region = test_region.T.reset_index()

In [119]:
test_region_fig = px.bar(test_region, x = ['Central','Western'] , y='index',barmode='group',range_x=[0,60],
                         color_discrete_sequence= discrete_color)
test_region_fig.update_xaxes(title='Proportion(%)',linecolor=linecolor)
test_region_fig.update_yaxes(title='Tests',title_font=titlefont)
test_region_fig.update_layout(margin=margin,legend=dict(orientation='h',title=None,yanchor='top',y=1.1,xanchor='right',x=1))


test_period_figure = heatmap_plot(test_period,'Tests') #FIGURE showing tests done at admission by period
##########################################################################################


## Recommended figures

## 1. Symptoms by Region Heatmap

In [243]:
# data[['record_id','month','region','cough','headache','diarrhoea','chest_pain','vomiting','fever','loss_of_consciousness','dysuria',
#       'difficulty_in_breathing','abdominal_pain','easy_fatigability']]




def process_symptoms(data, symptoms):
    data = data[['region', 'month'] + symptoms].replace('Empty', None)
    symptom_counts = {}
    
    for symptom in symptoms:
        filtered_data = data[data[symptom] == 'Yes'][['region', symptom]].groupby('region').count()
        symptom_counts[symptom] = filtered_data
    
    merged_data = pd.concat(symptom_counts.values(), axis=1, keys=symptom_counts.keys()).fillna(0)
    merged_data[f'{symptoms[0]}_{symptoms[-1]}'] = merged_data.sum(axis=1)
    return merged_data[[f'{symptoms[0]}_{symptoms[-1]}']]

symptom_categories = [
    ['difficulty_in_breathing', 'cough'],
    ['easy_fatigability', 'chest_pain'],
    ['vomiting', 'diarrhoea', 'abdominal_pain'],['headache','loss_of_consciousness'],['dysuria']
]

result = pd.DataFrame()

for category in symptom_categories:
    category_name = "_".join(category)
    symptom_data = process_symptoms(data, category)
    result = pd.concat([result, symptom_data], axis=1, join='outer')


symptoms = result.rename(columns={'difficulty_in_breathing_cough':'Respiratory','easy_fatigability_chest_pain':'Cardiovascular',
                                  'vomiting_abdominal_pain':'Gastrointestinal','headache_loss_of_consciousness':'Central_Nervous',
                                  'dysuria_dysuria':'Genitourinary'})
symptoms




,Respiratory,Cardiovascular,Gastrointestinal,Central_Nervous,Genitourinary
,,,,,
region,,,,,
Central,2912.0,1522.0,3385,1586,89
Western,3212.0,2649.0,5831,3431,294
Coast,56.0,32.0,108,39,1


In [227]:
data['diarrhoea']

0        None
1         Yes
2         Yes
3          No
4        None
         ... 
19386    None
19387    None
19388    None
19389      No
19390    None
Name: diarrhoea, Length: 17986, dtype: object

# Data Clarifications

How important is the Medical Admission Record form used?

There exists:

    1. 3 entries with no gender information.

    
    2. 4 entries with dates before 2023-01-01

    3. How do we extract region information? No variable to define region. Do we use hospital or subcounty?

    5. Why weight specified  Min: 40, Max: 150

    6. On the outcome section, what is D1_present?

    7. From current dataset 2 guys have no hospital information - 316-1057,318-234





In [22]:
data1 = pd.read_table('../../Kenya-COVID-19-Dashboard/syndromic_data/kenya_data/1689336148515.metadata.tsv')
data2 = pd.read_table('../../Kenya-COVID-19-Dashboard/syndromic_data/kenya_data/1689336178136.metadata.tsv')
data3 = pd.read_table('../../Kenya-COVID-19-Dashboard/syndromic_data/kenya_data/1689336349172.metadata.tsv')

data1 = data1[['strain','originating_lab','submitting_lab']]
data2 = data2[['strain','originating_lab','submitting_lab']]
data3 = data3[['strain','originating_lab','submitting_lab']]

df = pd.concat([data1,data2,data3])

In [25]:
len(df)

12630

In [24]:
df.to_csv('../syndromic_data/kenya_data/labs.txt',sep='\t',index=False)